# Gratias, Deo (2023)

In [ ]:
%load_ext abjad_notebook
import itertools
from pathlib import Path

import abjad
import librosa
import numpy as np
import plotly.express as px
import soundfile as sf
from IPython.display import Audio

from performer.composition.score import Renderer, parser

## Define constants

In [ ]:
SAMPLE_RATE = 48000
N_FFT = 1024
HOP_LENGTH = 512
OUTPUT_DIR = Path("../gratias-deo")
DEO_PATH = OUTPUT_DIR / "deo-original.wav"
ABJAD_INCLUDE_DIR = abjad.configuration.Configuration().abjad_directory / "abjad/scm"

FLT_CKPT = "../checkpoints/flute_longrun.ckpt"
VLN_CKPT = "../checkpoints/violin_longrun.ckpt"
VLC_CKPT = "../checkpoints/cello_longrun.ckpt"
DRM_CKPT = "../checkpoints/drums_baseline.ckpt"

## Some lily files to include

In [ ]:
abjad_ily = f'\\include "{ABJAD_INCLUDE_DIR}/abjad.ily"\n'

event_listener = r"""#(ly:set-option 'relative-includes #t)
\include "./event-listener.ly"
"""

## Helper functions

In [ ]:
def save_audio_file(file_path, data):
    sf.write(f"{OUTPUT_DIR}/{file_path}", data.T, SAMPLE_RATE, subtype="PCM_24")


def get_notes_file(pdf_path, instrument_name):
    notes_path = pdf_path.parent / f"{pdf_path.stem}-{instrument_name}.notes"

    return notes_path


def slur_parts(voice, counts):
    notes = abjad.select.notes(voice)
    parts = abjad.select.partition_by_counts(notes, counts, cyclic=True)
    for part in parts:
        first_note, last_note = part[0], part[-1]
        accent = abjad.Articulation("accent")
        start_slur = abjad.StartSlur()
        abjad.attach(accent, first_note)
        abjad.attach(start_slur, first_note)
        staccato = abjad.Articulation("staccato")
        stop_slur = abjad.StopSlur()
        abjad.attach(staccato, last_note)
        abjad.attach(stop_slur, last_note)


def tupletize_notes(voice, counts, modulus):
    notes = abjad.select.notes(voice)
    parts = abjad.select.partition_by_counts(notes, counts, cyclic=True)
    for i, part in enumerate(parts):
        if i % len(counts) == modulus:
            abjad.mutate.wrap(part, abjad.Tuplet("3:2"))


# The rhythm of the piece is based on even number fibonacci patterns concatenated
# with their mirror images.
def get_fibonacci_pattern(n=3):
    previous_pattern = [2]
    current_pattern = []
    while n >= 0:
        for m in previous_pattern:
            if m == 2:
                current_pattern.extend([2, 1])
            else:
                current_pattern.append(2)
        n = n - 1
        previous_pattern = current_pattern
        current_pattern = []

    return previous_pattern


def fibonacci_generator():
    pattern = [2, 1]
    step = -1

    while True:
        # yield elements of current pattern
        for e in pattern:
            yield e

        pattern = get_fibonacci_pattern(step)
        step += 1

In [ ]:
flute_renderer = Renderer(FLT_CKPT, "cpu")
drum_renderer = Renderer(DRM_CKPT)

## Load `Deo` sample

In [ ]:
deo_audio, sr = librosa.load(DEO_PATH, sr=SAMPLE_RATE, mono=False)

## Do STFT transform

In [ ]:
deo_complex_stft = librosa.stft(deo_audio, n_fft=N_FFT, hop_length=HOP_LENGTH)
deo_db_stft = librosa.amplitude_to_db(np.abs(deo_complex_stft), top_db=120)
deo_db_stft_mono = deo_db_stft.mean(axis=0)

fft_frequencies = librosa.fft_frequencies(sr=SAMPLE_RATE, n_fft=N_FFT)

## Pick most prominent STFT bins

In [ ]:
# Select bins with mean dB value greater than `db_threshold`
db_threshold = -24.0
mean_db_per_bin = deo_db_stft_mono.mean(axis=1)
criterion = mean_db_per_bin > db_threshold

selected_bins = np.where(criterion)[0]

# sort selected bins by mean dB
sorted_selected_bins = sorted(selected_bins, key=lambda x: mean_db_per_bin[x])

# manual sub selection
select_indices = [
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    12,
    13,
    14,
    15,
    17,
    18,
    19,
    20,
    21,
    22,
    24,
    27,
    28,
    29,
    30,
    31,
]
sorted_selected_bins = [sorted_selected_bins[idx] for idx in select_indices]

## Get a separate STFT  for each selected bin

In [ ]:
masked_complex_stfts = []

for idx in sorted_selected_bins:
    mask = np.zeros_like(deo_complex_stft)
    mask[:, idx, :] = 1
    masked_stft = deo_complex_stft * mask

    masked_complex_stfts.append(masked_stft)

## Generate melody, canon, and harmony audio files for each STFT

In [ ]:
canon_partials = []

for masked_complex_stft in masked_complex_stfts:
    y = librosa.istft(masked_complex_stft, hop_length=HOP_LENGTH)
    canon_partials.append(y)

# prepare sounds
canon = np.concatenate(np.cumsum(canon_partials, axis=0), axis=1)
melody = np.concatenate(canon_partials, axis=1)
harmony = np.sum(canon_partials, axis=0)

save_audio_file("deo-melody.wav", melody)
save_audio_file("deo-canon.wav", canon)
save_audio_file("deo-harmony.wav", harmony)

## Prepare the theme for Canon for 1 Voice

### Prepare notes, voice, staff

In [ ]:
# get note dynamics. we'll manually add dynamics and hairpins to score/
dynamics = np.array(["ppp", "pp", "p", "mp", "mf", "f", "ff", "fff"])

note_dynamics = mean_db_per_bin[sorted_selected_bins]
note_dynamics -= note_dynamics.min()
note_dynamics /= note_dynamics.max()

print(np.round(note_dynamics / (1 / 7)).astype("int"))

In [ ]:
note_names = librosa.hz_to_note(fft_frequencies[sorted_selected_bins])

abjad_notes = [abjad.Note.from_pitch_and_duration(n.replace("♯", "#"), (3, 4)) for n in note_names]

# Add quarter sharps to notes
quarter_sharps = (np.round(2 * librosa.hz_to_midi(fft_frequencies[sorted_selected_bins])) / 2) % 1
abjad_quarter_sharp = abjad.Accidental("quarter sharp")
for idx, qs in enumerate(quarter_sharps):
    if qs == 0:
        continue
    base_name = abjad_notes[idx].written_pitch.name[0]
    ticks = abjad_notes[idx].written_pitch.octave.ticks
    base_name += ticks
    new_accidental = abjad_notes[idx].written_pitch.accidental + abjad_quarter_sharp
    new_note = abjad.NamedPitch(base_name, accidental=new_accidental)
    abjad_notes[idx].written_pitch = new_note


canon_voice = abjad.Voice(abjad_notes, name="Voice_1")
canon_staff = abjad.Staff([canon_voice], name="Canon_1")

# add dynamics manually
abjad.hairpin("pp < p", canon_voice[:5])
abjad.hairpin("< mp", canon_voice[5:16])
abjad.hairpin("< f", canon_voice[16:20])
# abjad.hairpin("< ff", canon_voice[20:22])
abjad.hairpin("< ff", canon_voice[22:])
abjad.override(canon_voice[0]).DynamicLineSpanner.staff_padding = 4

# Add instrument name
instrument_name = "Flute"
markup = abjad.Markup(f'\\markup "{instrument_name}"')
abjad.attach(abjad.InstrumentName(markup), canon_voice[0])

# add ottava marks manually
abjad.ottava(canon_voice[:18], start_ottava=abjad.Ottava(2), stop_ottava=abjad.Ottava(0))

# calculate tempo. assume 3/4 time signature. harmony should be 1 bar long, so 3 quarter notes.
# we can find number of quarter notes per minute.
tempo = 60 * 3 * SAMPLE_RATE / harmony.shape[1]
print(f"BPM: {tempo:.2f}")

# add tempo mark. event listener requires integer tempo (I guess)
mark = abjad.MetronomeMark((1, 4), int(tempo))
abjad.attach(mark, canon_voice[0])

# add time signature
time_signature = abjad.TimeSignature((3, 4))
abjad.attach(time_signature, canon_voice[0])

# prepare and preview file
lilypond_file = abjad.LilyPondFile([event_listener, abjad_ily, canon_staff])
abjad.show(lilypond_file)

### Export event list, parse, and render

In [ ]:
pdf_path, *_ = abjad.persist.as_pdf(lilypond_file, OUTPUT_DIR / "gratias-deo.pdf")
pdf_path = Path(pdf_path)
notes_path = get_notes_file(pdf_path, instrument_name)

notes = parser(notes_path)
y = flute_renderer.render(notes)

Audio(data=y, rate=48000, normalize=False)

# Write human parts

In [ ]:
class Pitch:
    def __init__(self, name, accidental, octave):
        self.name = name
        self.accidental = accidental
        self.octave = octave

    @property
    def pitch(self):
        return abjad.NamedPitch(name=self.name, accidental=self.accidental, octave=self.octave)

In [ ]:
# take all to a single octave
pcs = []
for idx, n in enumerate(abjad_notes):
    base_name = abjad_notes[idx].written_pitch.name[0]
    accidental = abjad_notes[idx].written_pitch.accidental
    octave = 5
    pc = Pitch(base_name, accidental, octave)
    if pc not in pcs:
        pcs.append(pc)

In [ ]:
scale_voice = abjad.Voice([abjad.Note(pc.pitch, (1, 16)) for pc in pcs], name="Scale Voice")
scale_staff = abjad.Staff([scale_voice], name="Scale Staff")

# Add instrument name
instrument_name = "Flute II"
markup = abjad.Markup(f'\\markup "{instrument_name}"')
abjad.attach(abjad.InstrumentName(markup), scale_voice[0])

# add tempo mark. event listener requires integer tempo (I guess)
mark = abjad.MetronomeMark((1, 4), 130)
abjad.attach(mark, scale_voice[0])

# add time signature
time_signature = abjad.TimeSignature((4, 4))
abjad.attach(time_signature, scale_voice[0])

# add dynamics
abjad.attach(abjad.Dynamic("mp"), scale_voice[0])
abjad.override(scale_voice[0]).DynamicLineSpanner.staff_padding = 4


# adjust durations
notes = abjad.select.notes(scale_voice)
fibonacci = fibonacci_generator()
fib = [next(fibonacci) for i in range(len(notes))]
for n, f in zip(notes, fib):
    if f == 1:
        abjad.mutate.scale(n, abjad.Fraction(1, 2))

# add slurs
slur_parts(scale_voice, [3 if n == 1 else 5 for n in fib])

# prepare and preview file
lilypond_file = abjad.LilyPondFile([event_listener, abjad_ily, scale_staff])
abjad.show(lilypond_file)

In [ ]:
pdf_path, *_ = abjad.persist.as_pdf(lilypond_file, OUTPUT_DIR / "base-scale.pdf")
pdf_path = Path(pdf_path)
notes_path = get_notes_file(pdf_path, instrument_name)


notes = parser(notes_path)
y = flute_renderer.render(notes)

Audio(data=y, rate=48000, normalize=False)

In [ ]:
chords = abjad.select.partition_by_counts(pcs, [3, 5], cyclic=True)
phrases = sum([c * 5 for c in chords], start=[])
test_voice = abjad.Voice([abjad.Note(pc.pitch, (1, 16)) for pc in phrases], name="Test Voice")
test_staff = abjad.Staff([test_voice], name="Test Staff")

# Add instrument name
instrument_name = "Flute"
markup = abjad.Markup(f'\\markup "{instrument_name}"')
abjad.attach(abjad.InstrumentName(markup), test_voice[0])

# add tempo mark. event listener requires integer tempo (I guess)
mark = abjad.MetronomeMark((1, 4), 130)
abjad.attach(mark, test_voice[0])

# add time signature
time_signature = abjad.TimeSignature((3, 8))
abjad.attach(time_signature, test_voice[0])

# add dynamics
abjad.attach(abjad.Dynamic("mp"), test_voice[0])
abjad.override(test_voice[0]).DynamicLineSpanner.staff_padding = 4

# prepare and preview file
lilypond_file = abjad.LilyPondFile([event_listener, abjad_ily, test_staff])
abjad.show(lilypond_file)

In [ ]:
pdf_path, *_ = abjad.persist.as_pdf(lilypond_file, OUTPUT_DIR / "test-scale.pdf")
pdf_path = Path(pdf_path)
notes_path = get_notes_file(pdf_path, instrument_name)


notes = parser(notes_path)
y = flute_renderer.render(notes)

Audio(data=y, rate=48000, normalize=False)

In [ ]:
for n in notes.notes:
    n.f0 /= 2.0

In [ ]:
for n in notes.notes:
    n.f0 += np.random.randn(*n.f0.shape) * 100.0

In [ ]:
for n in notes.notes:
    n.envelope.gap_percent_duration = 0.5

In [ ]:
y = drum_renderer.render(notes)

Audio(data=y, rate=48000, normalize=False)